# SMOTE

> "작성 중"

- toc: true
- branch: master
- badges: true
- comments: true
- author: 한재수
- categories: [Machine Learning]

`-` ref: N. V. Chawla, K. W. Bowyer, L. O.Hall, W. P. Kegelmeyer, "SMOTE:
       synthetic minority over-sampling technique," Journal of artificial
       intelligence research, 321-357, 2002.

## SMOTE 작동 방식

`-` SMOTE를 사용하기 위해선 변수들이 모두 양적 변수여야 한다

`1.` 마이너 클래스의 각 샘플에 대하여 $k$개의 가까운 이웃을 계산한다

`2.` 마이너 클래스에서 $N$개의 샘플을 복원추출한다

`3.` 선택된 $N$개의 샘플 각각에 대하여 $k$개의 가까운 이웃중 무작위로 하나를 선택한다

`4.` 모든 변수에 대하여 샘플과 그 샘플의 이웃간의 내분점을 무작위로 선택한다  

### 예시